In [34]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
import joblib
from tensorflow.keras.layers import Normalization, StringLookup, IntegerLookup
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import TargetEncoder
from sklearn.compose import ColumnTransformer
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping

In [35]:
columns_to_drop = ['customer','merchant', 'ind_recommended', 'activation', 'label']
categorical_features = ['merchant_profile_01'] 
numeric_features =['customer_spend_06',
 'customer_digital_activity_16',
 'distance_05',
 'customer_digital_activity_19',
 'customer_industry_spend_03',
 'customer_digital_activity_20',
 'customer_spend_03',
 'merchant_spend_07',
 'customer_spend_18',
 'merchant_spend_03',
 'distance_02',
 'customer_profile_04',
 'customer_digital_activity_14',
 'customer_digital_activity_10',
 'customer_profile_02',
 'customer_spend_01',
 'customer_digital_activity_01',
 'merchant_spend_02',
 'distance_03',
 'customer_digital_activity_03',
 'merchant_spend_11',
 'customer_spend_02',
 'merchant_spend_01',
 'customer_digital_activity_21',
 'customer_merchant_03',
 'customer_spend_19',
 'distance_01',
 'customer_digital_activity_12',
 'customer_profile_01',
 'customer_digital_activity_13',
 'customer_spend_05',
 'customer_spend_16',
 'customer_spend_13',
 'merchant_profile_03',
 'merchant_spend_05',
 'customer_digital_activity_02',
 'customer_spend_17',
 'customer_digital_activity_06',
 'customer_digital_activity_15',
 'customer_spend_15',
 'merchant_spend_10',
 'distance_04',
 'merchant_spend_04',
 'customer_industry_spend_04',
 'customer_industry_spend_01',
 'merchant_profile_02',
 'customer_profile_03',
 'customer_digital_activity_17',
 'customer_digital_activity_11',
 'merchant_spend_09',
 'merchant_spend_08',
 'customer_industry_spend_02',
 'customer_spend_07',
 'customer_spend_14',
 'customer_digital_activity_05',
 'customer_spend_04',
 'customer_digital_activity_04',
 'merchant_spend_06',
 'customer_digital_activity_22',
 'customer_industry_spend_05']
X_cols = ['merchant_profile_01',
 'customer_spend_06',
 'customer_digital_activity_16',
 'distance_05',
 'customer_digital_activity_19',
 'customer_industry_spend_03',
 'customer_digital_activity_20',
 'customer_spend_03',
 'merchant_spend_07',
 'customer_spend_18',
 'merchant_spend_03',
 'distance_02',
 'customer_profile_04',
 'customer_digital_activity_14',
 'customer_digital_activity_10',
 'customer_profile_02',
 'customer_spend_01',
 'customer_digital_activity_01',
 'merchant_spend_02',
 'distance_03',
 'customer_digital_activity_03',
 'merchant_spend_11',
 'customer_spend_02',
 'merchant_spend_01',
 'customer_digital_activity_21',
 'customer_merchant_03',
 'customer_spend_19',
 'distance_01',
 'customer_digital_activity_12',
 'customer_profile_01',
 'customer_digital_activity_13',
 'customer_spend_05',
 'customer_spend_16',
 'customer_spend_13',
 'merchant_profile_03',
 'merchant_spend_05',
 'customer_digital_activity_02',
 'customer_spend_17',
 'customer_digital_activity_06',
 'customer_digital_activity_15',
 'customer_spend_15',
 'merchant_spend_10',
 'distance_04',
 'merchant_spend_04',
 'customer_industry_spend_04',
 'customer_industry_spend_01',
 'merchant_profile_02',
 'customer_profile_03',
 'customer_digital_activity_17',
 'customer_digital_activity_11',
 'merchant_spend_09',
 'merchant_spend_08',
 'customer_industry_spend_02',
 'customer_spend_07',
 'customer_spend_14',
 'customer_digital_activity_05',
 'customer_spend_04',
 'customer_digital_activity_04',
 'merchant_spend_06',
 'customer_digital_activity_22',
 'customer_industry_spend_05']

In [36]:
def predict(model, data):
  reconstructions = model(data)
  loss = tf.keras.losses.mae(reconstructions, data)
  # scaler = preprocessing.MinMaxScaler()
  # normalized_errors = scaler.fit_transform(loss.reshape(-1, 1))
  # params = beta.fit(normalized_errors)
  # probabilities = beta.pdf(normalized_errors, *params)
  return loss

In [37]:
class AnomalyDetector(Model):
  def __init__(self):
    super(AnomalyDetector, self).__init__()
    self.encoder = tf.keras.Sequential([  
        layers.Dense(32, activation="relu"),
        layers.Dense(16, activation="relu"),
      layers.Dense(8, activation="relu")])

    self.decoder = tf.keras.Sequential([
      layers.Dense(8, activation="relu"),
        layers.Dense(16, activation="relu"),
        layers.Dense(32, activation="relu"),
      layers.Dense(61, activation="sigmoid"),
    ])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

In [12]:
eval_df = pd.read_parquet('train_test/imputed_test.parquet')

In [13]:
# eval_df= eval_df.fillna(0)
# TO FILL NA

In [38]:
# Load the saved StandardScaler
preprocessor = joblib.load('scaler_model/model3_preprocessor1.joblib')

In [39]:
# Scale the remaining features using the same scaler used during training
x_new_scaled = preprocessor.transform(eval_df[X_cols])



In [40]:
len(x_new_scaled)

12604600

In [41]:
# Load the saved model
model =AnomalyDetector()
model.compile(optimizer='adam', loss='mae')   
model.load_weights('autoencoder_model1.weights.h5')

/home/FYP/poon0064/.conda/envs/RunJupyter/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 30 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


In [50]:

predictions = predict(model, x_new_scaled)
print(len(predictions))


12604600


In [55]:
predictions = predictions.numpy()

AttributeError: 'numpy.ndarray' object has no attribute 'numpy'

In [44]:
na_mask = np.isnan(predictions)

# Check if there are any NaN values and print the result
has_na = np.any(na_mask)
print("Contains NaN values:", has_na)

# If needed, count the number of NaN values
na_count = np.sum(na_mask)
print("Number of NaN values:", na_count)

Contains NaN values: False
Number of NaN values: 0


In [52]:
from scipy.stats import beta
where_are_NaNs = np.isnan(predictions)
predictions[where_are_NaNs] = 0
scaler = preprocessing.MinMaxScaler()
normalized_errors = scaler.fit_transform(predictions.reshape(-1, 1))
params = beta.fit(normalized_errors)
probabilities = beta.pdf(normalized_errors, *params)

/home/FYP/poon0064/.conda/envs/RunJupyter/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:723: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  a, b = optimize.fsolve(func, (1.0, 1.0))


In [56]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
normalized_errors = scaler.fit_transform(predictions.reshape(-1, 1))
predictions = np.abs(normalized_errors)

In [24]:
np.savetxt("predictions.csv", predictions, delimiter=",")

In [25]:
original_test_df = pd.read_csv('evaluation.csv')
customer_merchant_info = original_test_df[['customer', 'merchant']]

In [58]:
# Create a new DataFrame with columns A, B, and predictions
result_df = pd.concat([original_test_df[['customer','merchant']], pd.DataFrame(predictions, columns=['predicted_score'])], axis=1)

In [59]:
result_df[['customer', 'merchant', 'predicted_score']].to_csv('submission/final_submission3.csv', index=False)

In [ ]:
len(eval_df)